In [9]:
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as stopwords 
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.metrics import accuracy_score 
from sklearn.base import TransformerMixin 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
import extract_sentences
import pandas as pd
import numpy as np

In [2]:
import string
punctuations = string.punctuation

In [3]:
from spacy.lang.en import English
parser = English()

In [4]:
#Custom transformer using spaCy 
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]
    def fit(self, X, y=None, **fit_params):
        return self
    def get_params(self, deep=True):
        return {}

In [5]:
# Basic utility function to clean the text 
def clean_text(text):     
    return text.strip().lower()

In [6]:
#Create spacy tokenizer that parses a sentence and generates tokens
#these can also be replaced by word vectors 
def spacy_tokenizer(sentence):
    tokens = parser(sentence)
    tokens = [tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_ for tok in tokens]
    tokens = [tok for tok in tokens if (tok not in stopwords and tok not in punctuations)]     
    return tokens

#create vectorizer object to generate feature vectors, we will use custom spacy’s tokenizer
vectorizer = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
classifier = LinearSVC()

In [7]:
# Create the  pipeline to clean, tokenize, vectorize, and classify 
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', vectorizer),
                 ('classifier', classifier)])

In [8]:
# Load sample data
"""train = [('I love this sandwich.', 'pos'),          
         ('this is an amazing place!', 'pos'),
         ('I feel very good about these beers.', 'pos'),
         ('this is my best work.', 'pos'),
         ("what an awesome view", 'pos'),
         ('I do not like this restaurant', 'neg'),
         ('I am tired of this stuff.', 'neg'),
         ("I can't deal with this", 'neg'),
         ('he is my sworn enemy!', 'neg'),          
         ('my boss is horrible.', 'neg')] 
test =   [('the beer was good.', 'pos'),     
         ('I do not enjoy my job', 'neg'),
         ("I ain't feelin dandy today.", 'neg'),
         ("I feel amazing!", 'pos'),
         ('Gary is a good friend of mine.', 'pos'),
         ("I can't believe I'm doing this.", 'neg')]"""


In [31]:
# Load sample data
#manually identified/tagged synthesis paragraphs
train_p = [[117, 118, 119], [112], [117], [122, 125], [88]]
p=[1,2,3,4,5]
syn_yes=[]
syn_no=[]
for i in range(len(p)):
    sen_yes_arr, sen_no_arr = extract_sentences.extract_sentences('journal_articles/Paper' + str(p[i]) + '.html', train_p[p[i]-1])
    for j in range(len(sen_yes_arr)):
        syn_yes.append(sen_yes_arr[j])
    for k in range(len(sen_no_arr)):
        syn_no.append(sen_no_arr[k])
Syn_sen=pd.DataFrame({'x':syn_yes, 'y':np.ones(len(syn_yes))})
Syn_not_sen=pd.DataFrame({'x':syn_no, 'y':np.zeros(len(syn_no))})
Train=[Syn_sen,Syn_not_sen]
train_data=pd.concat(Train,ignore_index=True)
train_data

,x,y
0,We placed approximately 500 mg of CH3NH3I and ...,1.0
1,The device substrates were placed in a substra...,1.0
2,Once the pressure in the chamber was pumped do...,1.0
3,The substrate holder was rotated to ensure uni...,1.0
4,The substrate holder was water-cooled to appro...,1.0
5,Perovskite films were optimized for best devic...,1.0
6,"In particular, device performance was very sen...",1.0
7,In our trials we performed the following steps.,1.0
8,(1) We varied the as-deposited composition of ...,1.0
9,(2) We varied the film thickness—at the optimu...,1.0


In [34]:
train_data['x'][590]

Sentence('\xa0&\xa0Muhammad I. Haider', 0, 21)

In [30]:
for x in range(train_data.shape[0]):
    print(train_data['x'][x])

We placed approximately 500 mg of CH3NH3I and 100 mg of PbCl2 into separate crucibles.
The device substrates were placed in a substrate holder above the sources with the TiO2-coated FTO side facing down towards the sources.
Once the pressure in the chamber was pumped down to below 10−5 mbar, the two sources were heated slightly above their desired deposition temperatures for approximately 5 min (that is, CH3NH3I was heated to about 120 °C and PbCl2 was heated to about 325 °C) to remove volatile impurities before depositing the materials onto the substrate.
The substrate holder was rotated to ensure uniform coating throughout deposition, because the right-hand source predominantly coats the right-hand side of the substrate and similarly for the left.
The substrate holder was water-cooled to approximately 21 °C, though precise measurement of the substrate temperature during deposition was not performed.
Perovskite films were optimized for best device performance by varying the key deposi

Subsequently, the spiro-OMeTAD-based hole-transfer layer (80 mg spiro-OMeTAD, 28.5 µl 4-tert-butylpyridine and 17.5 µl lithium-bis(trifluoromethanesulfonyl)imide (Li-TFSI) solution (520 mg Li-TFSI in 1 ml acetonitrile) all dissolved in 1 ml chlorobenzene) was deposited by spin coating at 4,000 r.p.m. for 30 s.
Finally, a 150-nm-thick silver layer was deposited by thermal evaporation at a base pressure of 2 × 10−6 mbar.
The completed devices were stored in a N2-purged glovebox (<0.1 ppm O2 and H2O).
Prior to the evaporation of the Ag top contact, all device fabrication steps were carried out under ambient conditions.
Efficient planar heterojunction perovskite solar cells by vapour deposition | Nature
Cookie Notice
We use cookies to personalise content and ads, to provide social media features and to analyse our traffic.
We also share information about your use of our site with our social media, advertising and analytics partners in accordance with our Privacy Policy.
You can manage your

A diagram of the crystal structure is shown in Fig. 1d.
The main difference between CH3NH3PbI3 and the mixed-halide perovskite presented here is evident in a slight contraction of the c axis.
This is consistent with the Cl atoms in the mixed-halide perovskite residing in the apical positions, out of the PbI4 plane, as opposed to in the equatorial octahedral sites, as has been theoretically predicted18.
We now make a comparison between the thin-film topology and cross-sectional structure of devices fabricated by either vapour deposition or solution processing.
The top-view scanning electron microscope (SEM) images in Fig. 2a, b highlight the considerable differences between the film morphologies produced by the two deposition processes.
The vapour-deposited films are extremely uniform, with what appear to be crystalline features on the length scale of hundreds of nanometres.
In contrast, the solution-processed films appear to coat the substrate only partially, with crystalline ‘platelet

The vapour-deposited perovskite devices were fabricated on FTO-coated glass.
A compact layer of TiO2 was deposited on the FTO-coated glass by spin-coating (solution-processing) it with a mildly acidic solution of titanium isopropoxide in ethanol17, and subsequently the perovskite absorber was deposited on the compact TiO2-coated FTO substrate.
Methylammonium iodide (CH3NH3I) and lead chloride (PbCl2) were the organic and inorganic precursor salts, evaporated simultaneously from separate sources at 10−5 mbar with an as-deposited molar ratio of 4:1, based on the reading of the sensors above the crucibles.
A dark reddish-brown colour was observed immediately after evaporation.
Annealing the perovskite absorbers before spin-coating the hole-transporter layer fully crystallized the perovskite layer.
After spin-coating the hole transporter, spiro-OMeTAD, from a chlorobenzene solution to form the photoactive layer (including lithium bis(trifluoromethylsyfonyl)imide salt and tert-butylpyridine

Show context for reference 14
CAS
Article
Google Scholar
15.
Cheng, Z.
& Lin, J.
.
CrystEngComm 12, 2646–2662 (2010)
Show context for reference 15
CAS
Article
Google Scholar
16.
Heo, J.
H. et al.
.
Nature Photon.
7, 486–491 (2013)
Show context for reference 16
CAS
Article
Google Scholar
17.
Ball, J.
M., Lee, M.
M., Hey, A.
& Snaith, H.
J.
.
Energy Environ. Sci. 6, 1739–1743 (2013)
Show context for reference 17
CAS
Article
Google Scholar
18.
Mosconi, E., Amat, A., Nazeeruddin, M.
K., Grätzel, M.
& De Angelis, F.
.
J. Phys. Chem. C 117, 13902–13913 (2013)
Show context for reference 18
CAS
Article
Google Scholar
19.
Snaith, H.
J., Greenham, N.
C.
& Friend, R.
H.
.
Adv. Mater. 16, 1640–1645 (2004)
Show context for reference 19
CAS
Article
Google Scholar
20.
Yang, F., Shtein, M.
& Forrest, S.
R.
.
J. Appl. Phys. 98, 014906 (2005)
Show context for reference 20
CAS
Article
Google Scholar
21.
Sakata, H. et al.
.
(Conference Record of the Twenty-Eighth IEEE) 7–12 10.1109/PVSC.2000.915742 (2000)

...

The PCEs were averaged from the reverse (from the open-circuit voltage Voc to the short-circuit current Isc) and forward (from Isc to Voc) sweeps, because perovskite solar cells may experience a dependence of the measured efficiency on the scan direction.
However, the hysteresis of the cells fabricated from FAPbI3 and FAPbI3 incorporating 15 mol% MAPbBr3 was not relevant, even after collecting J−V curves with the relatively short scanning delay time of 40 ms (Fig. 1b).
In addition, the small discrepancies related to the scan direction for the FAPbI3/MAPbBr3 systems disappeared at longer delay times (above 100 ms).
That the hysteresis observed in FAPbI3-based systems is negligible compared to the hysteresis in MAPbI3 (Fig. 1b inset) may be related to the balance between electron and hole transport within the perovskite layer.
As reported, the electron-diffusion length in MAPbI3 (∼130 nm) is 1.4 times larger than the hole-diffusion length (∼90 nm)20, while FAPbI3 has a hole-diffusion len

The perovskite phase stabilization caused by the introduction of MAPbBr3 to FAPbI3 was also confirmed by an analysis carried out on the synthesized powders, which were prepared at room temperature by precipitation from solutions of FAPbI3, (FAPbI3)1 − x(MAPbI3)x, (FAPbI3)1 − x(FAPbBr3)x, and (FAPbI3)1 − x(MAPbBr3)x with x = 0.15.
Photographs of the as-prepared powders shown in Extended Data Fig. 4 indicate that a black powder like as-prepared MAPbI3 is obtained only for (FAPbI3)0.85(MAPbBr3)0.15.
In addition, the XRD spectra of the powders revealed that (FAPbI3)0.85(MAPbBr3)0.15, in contrast to FAPbI3, shows a pure perovskite phase (Extended Data Fig. 5), with no endothermic DSC peaks (Fig. 2a).
This finding confirms that the co-substitution of MA to FA and Br to I can efficiently stabilize the perovskite phase.
However, further investigation is required to determine the energetics of the perovskite and non-perovskite formation and to establish the composition of the stable form in per

EQE was measured by a power source (Newport 300W Xenon lamp, 66920) with a monochromator (Newport Cornerstone 260) and a multimeter (Keithley2001).
The J–V curves were measured using a solar simulator (Newport, Oriel Class A, 91195A) with a source meter (Keithley 2420) at 100 mA cm−2 AM1.5G illumination and a calibrated Si-reference cell certified by the National Renewable Energy Laboratory, USA.
The J–V curves were measured by reverse scan (forward bias (1.2 V) → short circuit (0 V)) or forward scan (short circuit (0 V) → forward bias (1.2 V)).
The step voltage was fixed at 10 mV and the delay time, which is a delay set at each voltage step before measuring each current, was modulated.
The J–V curves for all devices were measured by masking the active area with a metal mask (area of 0.096 cm2).
1.
Lee, M.
M. et al.
.
Science 338, 643–647 (2012)
Show context for reference 1
CAS
PubMed
Article
Google Scholar
2.
Kim, H.-S.
et al.
.
Sci. Rep. 2, 1–7 (2012)
Show context for reference 2
CAS

Burschka, J. et al.
.
Nature 499, 316–319 (2013)
CAS
PubMed
Article
Google Scholar
Liu, M., Johnston, M.
B.
& Snaith, H.
J.
.
Nature 501, 395–398 (2013)
CAS
PubMed
Article
Google Scholar
Noh, J.
H., Im, S.
H., Heo, J.
H., Mandal, T.
N.
& Seok, S.
I.
.
Nano Lett. 13, 1764–1769 (2013)
CAS
PubMed
Article
Google Scholar
Ball, J.
M., Lee, M.
M., Hey, A.
& Snaith, H.
J.
.
Energy Environ. Sci. 6, 1739–1743 (2013)
CAS
Article
Google Scholar
Jeon, N.
J. et al.
.
J. Am. Chem. Soc. 136, 7837–7840 (2014)
CAS
PubMed
Article
Google Scholar
Ryu, S. et al.
.
Energy Environ. Sci. 7, 2614–2618 (2014)
CAS
Article
Google Scholar
Malinkiewicz, O. et al.
.
Nature Photon.
8, 128–132 (2014)
CAS
Article
Google Scholar
Lee, J.-W. et al.
.
Adv. Mater. 26, 4991–4998 (2014)
CAS
PubMed
Article
Google Scholar
Zhou, H. et al.
.
Science 345, 542–546 (2014)
CAS
PubMed
Article
Google Scholar
Jeon, N.
J. et al.
.
Nature Mater. 13, 897–903 (2014)
CAS
Article
Google Scholar
Snaith, H.
J. et al.
.
J. Phys. Chem. Lett. 5, 15

Gary Hodes
 & David Cahen

sections

figures

references


Abstract


Main


Methods

Methods


References


Acknowledgements


Author information


Supplementary information


Rights and permissions


About this article


Further reading


Comments
Park, S.
H. et al.
.
Nature Photon.
3, 297–302 (2009).
CAS
Article
Google Scholar
Chen, H-Y.
et al.
.
Nature Photon.
3, 649–653 (2009).
CAS
Article
Google Scholar
Guo, X. et al.
.
Nature Photon.
7, 825–833 (2013).
CAS
Article
Google Scholar
Hardin, B.
E., Snaith, H.
J.
& McGehee, M.
D.
.
Nature Photon.
6, 162–169 (2012).
CAS
Article
Google Scholar
Yella, A. et al.
.
Science 334, 629–634 (2011).
CAS
PubMed
Article
Google Scholar
Park, N-G.
.
J. Phys. Chem. Lett. 4, 2423–2429 (2013).
CAS
Article
Google Scholar
Kojima, A., Teshima, K., Shirai, Y.
& Miyasaka, T.
.
J. Am. Chem. Soc. 131, 6050–6051 (2009).
CAS
PubMed
Article
Google Scholar
Zhao, Y.
& Zhu, K.
.
J. Phys. Chem. Lett. 4, 2880–2884 (2013).
CAS
Article
Google Scholar
Im, J-H., Lee, C-R

In [51]:
t=[0]
test_p = [[109]]
syn_test_yes=[]
syn_test_no=[]
for i in range(len(t)):
    sen_yes_arr, sen_no_arr = extract_sentences.extract_sentences('journal_articles/Paper' + str(t[i]) + '.html', test_p[t[i]])
    for j in range(len(sen_yes_arr)):
        syn_test_yes.append(sen_yes_arr[j])
    for k in range(len(sen_no_arr)):
        syn_test_no.append(sen_no_arr[k])
Syn_test_sen=pd.DataFrame({'X':syn_test_yes, 'Y':np.ones(len(syn_test_yes))})
Syn_test_not_sen=pd.DataFrame({'X':syn_test_no, 'Y':np.zeros(len(syn_test_no))})
Test=[Syn_test_sen,Syn_test_not_sen]
test_data=pd.concat(Test,ignore_index=True)
test_data

,X,Y
0,"A dense blocking layer of TiO2 (bl-TiO2, ∼70 n...",1.0
1,A 200–300-nm-thick mesoporous TiO2 (particle s...,1.0
2,CH3NH3I (MAI) and CH3NH3Br (MABr) were first s...,1.0
3,The precipitate was recovered by evaporation a...,1.0
4,"The prepared MAI and MABr powders, PbI2 (Aldri...",1.0
5,The resulting solution was coated onto the mp-...,1.0
6,"During the second spin-coating step, the subst...",1.0
7,A detailed time-rotation profile for the spin-...,1.0
8,The substrate was dried on a hot plate at 100 ...,1.0
9,"A solution of poly(triarylamine) (15 mg, PTAA,...",1.0


In [82]:
X_train=[str(train_data['x'][x]) for x in range(train_data.shape[0])]
Y_train=[str(train_data['y'][x]) for x in range(train_data.shape[0])]
X_test=[str(test_data['X'][x]) for x in range(test_data.shape[0])]
Y_test=[str(test_data['Y'][x]) for x in range(test_data.shape[0])]
X_test

['A dense blocking layer of TiO2 (bl-TiO2, ∼70 nm in thickness) was deposited onto a F-doped SnO2 (FTO, Pilkington, TEC8) substrate by spray pyrolysis, using a 20 mM titanium diisopropoxide bis(acetylacetonate) solution (Aldrich) at 450 °C to prevent direct contact between the FTO and the hole-conducting layer.',
 'A 200–300-nm-thick mesoporous TiO2 (particle size: about 50 nm, crystalline phase: anatase) film was spin-coated onto the bl-TiO2/FTO substrate using home-made pastes14 and calcining at 500 °C for 1 h in air to remove organic components.',
 'CH3NH3I (MAI) and CH3NH3Br (MABr) were first synthesized by reacting 27.86 ml CH3NH2 (40% in methanol, Junsei Chemical) and 30 ml HI (57 wt% in water, Aldrich) or 44 ml HBr (48 wt% in water, Aldrich) in a 250 ml round-bottom flask at 0 °C for 4 h with stirring, respectively.',
 'The precipitate was recovered by evaporation at 55 °C for 1 h. MAI and MABr were dissolved in ethanol, recrystallized from diethyl ether, and dried at 60 °C in a

In [81]:
text=X_test[0]
clean_text(text)

'a dense blocking layer of tio2 (bl-tio2, ∼70 nm in thickness) was deposited onto a f-doped sno2 (fto, pilkington, tec8) substrate by spray pyrolysis, using a 20 mm titanium diisopropoxide bis(acetylacetonate) solution (aldrich) at 450 °c to prevent direct contact between the fto and the hole-conducting layer.'

<class 'str'>
<class 'str'>


In [79]:
a=clean_text(text)
a

'we placed approximately 500\u2009mg of ch3nh3i and 100\u2009mg of pbcl2 into separate crucibles.'

In [102]:
# Create model and measure accuracy
a=pipe.fit([X_train[i] for i in range(len(X_train))], [Y_train[i] for i in range(len(Y_train))])
pred_data = pipe.predict([X_test[i] for i in range(len(X_test))]) 
for (sample, pred) in zip(X_test, pred_data):
    print (sample, pred) 
print ("Accuracy:", accuracy_score([Y_test[i] for i in range(len(Y_test))], pred_data))

A dense blocking layer of TiO2 (bl-TiO2, ∼70 nm in thickness) was deposited onto a F-doped SnO2 (FTO, Pilkington, TEC8) substrate by spray pyrolysis, using a 20 mM titanium diisopropoxide bis(acetylacetonate) solution (Aldrich) at 450 °C to prevent direct contact between the FTO and the hole-conducting layer. 0.0
A 200–300-nm-thick mesoporous TiO2 (particle size: about 50 nm, crystalline phase: anatase) film was spin-coated onto the bl-TiO2/FTO substrate using home-made pastes14 and calcining at 500 °C for 1 h in air to remove organic components. 1.0
CH3NH3I (MAI) and CH3NH3Br (MABr) were first synthesized by reacting 27.86 ml CH3NH2 (40% in methanol, Junsei Chemical) and 30 ml HI (57 wt% in water, Aldrich) or 44 ml HBr (48 wt% in water, Aldrich) in a 250 ml round-bottom flask at 0 °C for 4 h with stirring, respectively. 1.0
The precipitate was recovered by evaporation at 55 °C for 1 h. MAI and MABr were dissolved in ethanol, recrystallized from diethyl ether, and dried at 60 °C in a v

Generally, γ-butyrolactone (GBL), N,N-dimethylformamide, DMSO and N-methyl-2-pyrrolidone are used as effective solvents for lead halides and MAI. 0.0
However, simple spin-coating did not yield a homogeneous perovskite layer having uniform thickness over a large area20, although the convective spreading flow due to centrifugal force was applied to the slowly evaporating solvents. 0.0
Previously, we reported the formation of a pillared structure with an island-type upper layer10, and proposed the importance of a homogeneous upper layer on the mp-TiO2 base; however, we were unable to deposit uniform thin films by the solution process. 0.0
Furthermore, it was reported that the uniformity of the perovskite films depended on the thickness of the TiO2 compact layer, and modification of the spinning conditions could not achieve 100% surface coverage20. 1.0
This limited the applicability of the solution process. 0.0
Here, we report the solution-process fabrication of efficient perovskite solar 

The FTIR spectra of MAI, PbI2, DMSO–PbI2–DMSO, MAI–PbI2–DMSO and MAPbI3 are provided in Supplementary Fig. 3 for functional group comparison. 0.0
The detection of both the N–H and S–O deformation modes in the intermediate phase guarantees the successful inclusion of a solvent molecule (DMSO) and MAI into the PbI2. 0.0
Furthermore, the C=C mode indicative of the toluene dripping solvent is not detected. 0.0
The conversion of the intermediate phase into perovskite with annealing temperature was monitored by in situ high-temperature XRD. 0.0
The low-angle diffraction peaks in the XRD pattern of Fig. 2c clearly show that, with increasing temperature, the putative MAI–PbI2–DMSO disappears and new peaks are observed due to the appearance of MAPbI3 perovskite. 0.0
We see that the formation of the perovskite phase is accompanied by the complete transformation of the MAI–PbI2–DMSO at 130 °C, whereas both MAI–PbI2–DMSO and perovskite phases coexist at 100 °C. 1.0
On the basis of the above observ

Interestingly, the efficiency measured in the reverse scan did not vary greatly with the mp-TiO2 thickness, whereas the forward scan yielded a pronounced variation, resulting in a large discrepancy between the two scan directions. 0.0
Such a strong dependence of the measured efficiency on the scan direction with mp-TiO2 thickness suggests that solar cells fabricated from MAPbX3 perovskite materials operate differently than other types of cells such as organic, dye-sensitized, and conventional Si solar cells. 0.0
The stepwise increase or decrease in the bias voltage, corresponding to a reverse or forward scan, respectively, requires a charge redistribution to reach a new equilibrium state. 0.0
These characteristics of perovskite solar cells may be related to the large diffusion capacitance of such cells operating under reverse or forward biases24. 0.0
The large capacitance is assumed to be related to the underestimation in the forward scan and overestimation in the reverse scan by charg

PubMed 0.0
Article 0.0
Google Scholar 0.0
12. 0.0
Liu, M., Johnston, M. 0.0
B. 0.0
& Snaith, H. 0.0
J. 0.0
. 0.0
Nature 501, 395–398 (2013). 0.0
Show context for reference 12 0.0
CAS 0.0
PubMed 0.0
Article 0.0
Google Scholar 0.0
13. 0.0
Noh, J. 0.0
H., Im, S. 0.0
H., Heo, J. 0.0
H., Mandal, T. 0.0
N. 0.0
& Seok, S. 0.0
I. 0.0
. 0.0
Nano Lett. 13, 1764–1769 (2013). 0.0
Show context for reference 13 0.0
CAS 0.0
PubMed 0.0
Article 0.0
Google Scholar 0.0
14. 0.0
Malinkiewicz, O. et al. 0.0
. 0.0
Nature Photon. 0.0
6, 128–132 (2014). 0.0
Show context for reference 14 0.0
Google Scholar 0.0
15. 0.0
Waleed, W. 0.0
A. 0.0
& Etgar, L. 0.0
. 0.0
Energy Environ. Sci. 6, 3249–3253 (2013). 0.0
Show context for reference 15 0.0
CAS 0.0
Article 0.0
Google Scholar 0.0
16. 0.0
You, J. et al. 0.0
. 0.0
ACS Nano 8, 1674–1680 (2014). 0.0
Show context for reference 16 0.0
CAS 0.0
PubMed 0.0
Article 0.0
Google Scholar 0.0
17. 0.0
Stranks, S. 0.0
D. et al. 0.0
. 0.0
Science 342, 341–344 (2013). 0.0
Show cont

Article 0.0
Google Scholar 0.0
Xing, G. et al. 0.0
. 0.0
Science 342, 344–347 (2013). 0.0
CAS 0.0
PubMed 0.0
Article 0.0
Google Scholar 0.0
Jeng, J-U. 0.0
et al. 0.0
. 0.0
Adv. Mater. 25, 3727–3732 (2013). 0.0
CAS 0.0
PubMed 0.0
Article 0.0
Google Scholar 0.0
Eperon, G. 0.0
E., Burlakov, V. 0.0
M., Docampo, P., Goriely, A. 0.0
& Snaith, H. 0.0
J. 0.0
. 0.0
Adv. Funct. Mater. 24, 151–157 (2014). 0.0
CAS 0.0
Article 0.0
Google Scholar 0.0
Beckmann, P. 0.0
A. 0.0
. 0.0
Cryst. Res. Technol. 45, 455–460 (2010). 0.0
CAS 0.0
Article 0.0
Google Scholar 0.0
Miyamae, H. et al. 0.0
. 0.0
Chem. Lett. 6, 663–664 (1980). 0.0
Article 0.0
Google Scholar 0.0
Herman, M. et al. 0.0
. 0.0
Int. J. 0.0
Photoenergy 2012, 151452 (2012) 0.0
Article 0.0
Google Scholar 0.0
Koide, N. 0.0
& Han, I. 0.0
. 0.0
Rev. Sci. Instrum. 75, 2828–2831 (2004). 0.0
CAS 0.0
Article 0.0
Google Scholar 0.0
ISSN 1476-4660 (online) 0.0
About us 0.0
Press releases 0.0
Press office 0.0
Contact us 0.0
Journals A-Z 0.0
Articles by subj

In [117]:
y = pred_data.astype(np.float)
y

array([0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [118]:
Synthesis_sentence=[]
Not_Synthesis_sentence=[]
for i in range(len(y)):
    if y[i]==1:
        Synthesis_sentence.append(X_test[i])
    else:
        Not_Synthesis_sentence.append(X_test[i]) 

In [119]:
Synthesis_sentence

['A 200–300-nm-thick mesoporous TiO2 (particle size: about 50 nm, crystalline phase: anatase) film was spin-coated onto the bl-TiO2/FTO substrate using home-made pastes14 and calcining at 500 °C for 1 h in air to remove organic components.',
 'CH3NH3I (MAI) and CH3NH3Br (MABr) were first synthesized by reacting 27.86 ml CH3NH2 (40% in methanol, Junsei Chemical) and 30 ml HI (57 wt% in water, Aldrich) or 44 ml HBr (48 wt% in water, Aldrich) in a 250 ml round-bottom flask at 0 °C for 4 h with stirring, respectively.',
 'The precipitate was recovered by evaporation at 55 °C for 1 h. MAI and MABr were dissolved in ethanol, recrystallized from diethyl ether, and dried at 60 °C in a vacuum oven for 24 h.',
 'The resulting solution was coated onto the mp-TiO2/bl-TiO2/FTO substrate by a consecutive two-step spin-coating process at 1,000 and 5,000 r.p.m for 10 and 20 s, respectively.',
 'During the second spin-coating step, the substrate (around 1 cm × 1 cm) was treated with toluene drop-castin

[]